In [ ]:
#TODO: Table ppm for top 30
#TODO: Table OTD for top 30
#TODO: Table ppm for "critical" suppliers
#TODO: Table OTD for "critical" suppliers
#TODO: Table ppm for all
#TODO: Table OTD for all
#TODO: ppm KPI# for MDM, and ppm TTM 
#TODO: "critical" suppliers with NCRs

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import pyodbc
import numpy as np
import docx
import os
import seaborn as sns
sns.set(color_codes=True)

In [3]:
# variables
# get current time for sql where statement
cur_time = dt.datetime.now()

#rpt_date = "2018-01"
rpt_date_year = input('Enter year for report. (ex. "2018"): ')
rpt_date_month = input('Enter month for report. (ex. "01"): ')
rpt_date = rpt_date_year + '-' + rpt_date_month

# vendor number
vendor_num = 1001

Enter year for report. (ex. "2018"): 2018
Enter month for report. (ex. "01"): 03


In [4]:
# get receipts data
# production env
# connect to SQL
db = 'V&M_Optimus_Extracts'
connection = pyodbc.connect('Driver={SQL Server}; Server=okcpsql01p; Database=' + db + '; uid=; pwd=')
cursor = connection.cursor()

sql_cmd = """
    SELECT *
    FROM [V&M_Optimus_Extracts].[dbo].[VP_A201_ZMHIST_VendorOTD]
    WHERE [Posting Date] > '2017-01-01' and [Posting Date] < '2018-04-01'
"""

df_receipts = pd.read_sql(sql_cmd, connection)

df_receipts.dtypes

Posting Date            datetime64[ns]
Material                        object
Material description            object
Qty                            float64
AmountLC                       float64
Customer                        object
Customer Name                   object
Delivery                        object
Period                          object
Mat. Doc.                       object
MvT                             object
MRPC                            object
Plnt                            object
PGr                             object
RecWeight                      float64
Ref.doc.                        object
QtyPM                          float64
Value                          float64
Unit Cost                       object
User Name                       object
OKC_MRP                         object
MPS_Product                     object
Detailed Product                object
Inventory Type                  object
Net Gross Weight               float64
Net Weight               

In [5]:
df_receipts.to_excel('receipts.xlsx')

In [7]:
df_receipts.head()

,Purchasing Document(EBELN),Item(EBELP),Material(EMATN),Material Description,Net Net Weight,Net Gross Weight,Scheduled Quantity,Delivered Quantity,Open Qty,PO Creation Date,...,NUMBER OF DLV DATE CHANGES,DaysDiff(DeliveryDate-PO_OAD),DaysDiff(NeedDate-DeliveryDate),Exception Message,ComponentType,Size,Made of Material,Pressure,RF/RJ,CAST
0,4511817666,00010,038315-23-02,"FITTING, BODY VENT, STANDARD HEX HEAD",640.0,640.0,1600.0,0.0,1600.0,2018-02-19,...,0,0,-13,Reschedule In,FITTING,,,,,
1,4511817666,00020,038315-23-05,"FITTING, BODY VENT, STANDARD HEX HEAD",600.0,600.0,2000.0,0.0,2000.0,2018-02-19,...,0,0,0,No Change,FITTING,,,,,
2,4511818021,00010,038315-23-05,"FITTING, BODY VENT, STANDARD HEX HEAD",600.0,600.0,2000.0,0.0,2000.0,2018-02-19,...,0,0,0,No Change,FITTING,,,,,
3,4511787513,00010,038315-23-05,"FITTING, BODY VENT, STANDARD HEX HEAD",600.0,600.0,2000.0,0.0,2000.0,2018-01-24,...,0,0,0,No Change,FITTING,,,,,
4,4511831654,00010,038315-23-05T,"FITTING, BODY VENT, STANDARD HEX HEAD",540.0,540.0,1800.0,0.0,1800.0,2018-03-01,...,0,0,0,No Change,None,None,None,None,None,None


In [140]:
df_receipts['Posting Date'].describe()

count                   42872
unique                    326
top       2017-09-12 00:00:00
freq                      320
first     2017-01-03 00:00:00
last      2018-03-06 00:00:00
Name: Posting Date, dtype: object

In [151]:
df_receipts.dtypes

Posting Date            datetime64[ns]
Material                        object
Material description            object
Qty                            float64
AmountLC                       float64
Customer                        object
Customer Name                   object
Delivery                        object
Period                          object
Mat. Doc.                       object
MvT                             object
MRPC                            object
Plnt                            object
PGr                             object
RecWeight                      float64
Ref.doc.                        object
QtyPM                          float64
Value                          float64
Unit Cost                       object
User Name                       object
OKC_MRP                         object
MPS_Product                     object
Detailed Product                object
Inventory Type                  object
Net Gross Weight               float64
Net Weight               

In [153]:
def on_time(col):
    if col < 1:
        return True
    else:
        return False

In [177]:
df_receipts['on_time'] = df_receipts["Days Diff"].apply(on_time)
vendor_otd_full = '{:.2%}'.format(1 - df_receipts['on_time'].value_counts()[0]/df_receipts['on_time'].value_counts()[1])
vendor_otd_full

'71.19%'

In [179]:
# create search column
df_receipts['vendor_search'] = df_receipts['Vendor'] + ' ' + df_receipts['Vendor Name']
df_receipts['vendor_search']

0                                 10037535 CAMPEX INC
1                                 10037535 CAMPEX INC
2                                 10037535 CAMPEX INC
3                                 10037535 CAMPEX INC
4                                 10037535 CAMPEX INC
5                                 10037535 CAMPEX INC
6                                 10037535 CAMPEX INC
7                                 10037535 CAMPEX INC
8                                 10037535 CAMPEX INC
9                                 10037535 CAMPEX INC
10                                10037535 CAMPEX INC
11                                10037535 CAMPEX INC
12                                10037535 CAMPEX INC
13                                10037535 CAMPEX INC
14                                                   
15                                                   
16                      10005082 QUALITY MACHINE SHOP
17                      10005082 QUALITY MACHINE SHOP
18                      1000

In [180]:
# list all vendors with receipts
vendors = df_receipts.vendor_search.unique()
vendors

array(['10037535 CAMPEX INC', '         ', '10005082 QUALITY MACHINE SHOP',
       '10014845 TEAMCO INDUSTRIES CORP', '10070997 WON ZONE CORP',
       '10009158 TURPEN AND ASSOCIATES INC',
       '10162451 OKLAHOMA CUSTOM COATING', '10074507 NINGBO RONGLI METAL',
       '10074507 NINGBO YINZHOU RONGLI METAL',
       '10008245 BALLCO MANUFACTURING INC', '10021941 ACM MACHINING INC',
       '10080868 NEW ERA MAXTEK INC', '10011158 PARKER SERVICE CENTER',
       '10012505 FLUOROCARBON COMPANY LTD',
       '10024251 SOUTHEAST MACHINE INC', '10107300 COLLISON-GOLL LTD',
       'VA216    CAMERON CANADA CORP', '10132563 STANDARD HORSE NAIL CORP',
       '10059584 FLOW LINK SYSTEMS PVT LTD',
       '10034889 ALL-PRO FASTENERS INC', '10012446 FLUOROCARBON CO LTD',
       '10009090 MAYCO INC', '10002170 ENGINEERED SPRING PRODUCTS INC',
       '10045102 D AND J TECHNOLOGIES INC',
       '10038187 WESTERN HEAT AND FORGE PVT LTD',
       '10036283 TMS INDIA PVT LTD', '10075331 LONE STAR FASTENERS L

In [181]:
df_vendors = pd.DataFrame.from_dict(vendors)
df_vendors.columns = ['Vendor']
df_vendors

,Vendor
0,10037535 CAMPEX INC
1,
2,10005082 QUALITY MACHINE SHOP
3,10014845 TEAMCO INDUSTRIES CORP
4,10070997 WON ZONE CORP
5,10009158 TURPEN AND ASSOCIATES INC
6,10162451 OKLAHOMA CUSTOM COATING
7,10074507 NINGBO RONGLI METAL
8,10074507 NINGBO YINZHOU RONGLI METAL
9,10008245 BALLCO MANUFACTURING INC


In [182]:
df_vendors['vendor_id'] = df_vendors['Vendor'].str[:8]
df_vendors

,Vendor,vendor_id
0,10037535 CAMPEX INC,10037535
1,,
2,10005082 QUALITY MACHINE SHOP,10005082
3,10014845 TEAMCO INDUSTRIES CORP,10014845
4,10070997 WON ZONE CORP,10070997
5,10009158 TURPEN AND ASSOCIATES INC,10009158
6,10162451 OKLAHOMA CUSTOM COATING,10162451
7,10074507 NINGBO RONGLI METAL,10074507
8,10074507 NINGBO YINZHOU RONGLI METAL,10074507
9,10008245 BALLCO MANUFACTURING INC,10008245


In [183]:
# search for vendor number by name
search_term = input('Enter text to search for: ').upper()
df_vendor_single = df_vendors[df_vendors['Vendor'].str.contains(search_term)]
df_vendor_single

Enter text to search for: campex


,Vendor,vendor_id
0,10037535 CAMPEX INC,10037535


In [184]:
vendor_code = input('Enter vendor number (8 digits): ')
df_receipts_single = df_receipts[df_receipts['Vendor'].str.contains(vendor_code)]
df_receipts_single

Enter vendor number (8 digits): 10037535


,Posting Date,Material,Material description,Qty,AmountLC,Customer,Customer Name,Delivery,Period,Mat. Doc.,...,Trunnion Tonnage,Tons,Deliv.date,Stat-RelDelDate,Days Diff,PO Number,Category,Indirect,vendor_search,on_time
0,2017-01-13,5A-1020101532600,"CAP,1-1/2"" FP 6000 WOG FNPT,ASTM A350 (L",150.0,1866.00,,,,2017/01,5003562986,...,,0.20,01/20/2017,2017-02-14,-32.0,4511262933,> 3 Days Early,,10037535 CAMPEX INC,True
1,2017-03-21,5A-1020101532600,"CAP,1-1/2"" FP 6000 WOG FNPT,ASTM A350 (L",100.0,1244.00,,,,2017/03,5003732811,...,,0.14,03/27/2017,2017-04-21,-31.0,4511326344,> 3 Days Early,,10037535 CAMPEX INC,True
2,2017-02-21,5A-1020101532600,"CAP,1-1/2"" FP 6000 WOG FNPT,ASTM A350 (L",100.0,1244.00,,,,2017/02,5003661843,...,,0.14,02/24/2017,2017-03-21,-28.0,4511301306,> 3 Days Early,,10037535 CAMPEX INC,True
3,2017-04-25,5A-1020101532600,"CAP,1-1/2"" FP 6000 WOG FNPT,ASTM A350 (L",100.0,1244.00,,,,2017/04,5003825621,...,,0.14,05/05/2017,2017-06-06,-42.0,4511381382,> 3 Days Early,,10037535 CAMPEX INC,True
4,2017-04-17,5A-1020101532600,"CAP,1-1/2"" FP 6000 WOG FNPT,ASTM A350 (L",100.0,1244.00,,,,2017/04,5003802574,...,,0.14,04/24/2017,2017-05-09,-22.0,4511352434,> 3 Days Early,,10037535 CAMPEX INC,True
5,2017-04-17,5A-1020101532600,"CAP,1-1/2"" FP 6000 WOG FNPT,ASTM A350 (L",100.0,1244.00,,,,2017/04,5003802576,...,,0.14,04/24/2017,2017-05-22,-35.0,4511359080,> 3 Days Early,,10037535 CAMPEX INC,True
6,2018-03-06,5A-1020101532600,"CAP,1-1/2"" FP 6000 WOG FNPT,ASTM A350 (L",100.0,1188.00,,,,2018/03,5004671458,...,,0.14,03/07/2018,2018-03-07,-1.0,4511635888,ONTIME,,10037535 CAMPEX INC,True
7,2018-03-06,5A-1020101532600,"CAP,1-1/2"" FP 6000 WOG FNPT,ASTM A350 (L",101.0,1199.88,,,,2018/03,5004671459,...,,0.14,03/07/2018,2018-03-07,-1.0,4511635889,ONTIME,,10037535 CAMPEX INC,True
8,2018-03-06,5A-1020101532600,"CAP,1-1/2"" FP 6000 WOG FNPT,ASTM A350 (L",200.0,2376.00,,,,2018/03,5004671484,...,,0.27,03/07/2018,2018-03-07,-1.0,4511706490,ONTIME,,10037535 CAMPEX INC,True
9,2018-03-06,5A-1020101532600,"CAP,1-1/2"" FP 6000 WOG FNPT,ASTM A350 (L",200.0,2376.00,,,,2018/03,5004671487,...,,0.27,03/07/2018,2018-03-07,-1.0,4511713759,ONTIME,,10037535 CAMPEX INC,True


In [185]:
df_receipts_single['Posting Date'].describe()

count                    2796
unique                     87
top       2017-06-28 00:00:00
freq                      135
first     2017-01-09 00:00:00
last      2018-03-06 00:00:00
Name: Posting Date, dtype: object

In [149]:
df_receipts_single.columns

Index(['Posting Date', 'Material', 'Material description', 'Qty', 'AmountLC',
       'Customer', 'Customer Name', 'Delivery', 'Period', 'Mat. Doc.', 'MvT',
       'MRPC', 'Plnt', 'PGr', 'RecWeight', 'Ref.doc.', 'QtyPM', 'Value',
       'Unit Cost', 'User Name', 'OKC_MRP', 'MPS_Product', 'Detailed Product',
       'Inventory Type', 'Net Gross Weight', 'Net Weight', 'Size', 'Pressure',
       'Made of Material', 'CAST', 'RF/RJ', 'Post_Period', 'Week', 'Year',
       'Vendor', 'Vendor Name', 'ComponentType', 'Trunnion Tonnage', 'Tons',
       'Deliv.date', 'Stat-RelDelDate', 'Days Diff', 'PO Number', 'Category',
       'Indirect', 'vendor_search'],
      dtype='object')

In [188]:
df_receipts_single['on_time'].value_counts()

True     2511
False     285
Name: on_time, dtype: int64

In [189]:
total_rcpt = df_receipts_single['Qty'].sum()
'{:,.0f}'.format(total_rcpt)

'212,219'

In [190]:
vendor_otd_sing_full = '{:.2%}'.format(1 - df_receipts_single['on_time'].value_counts()[0]/df_receipts_single['on_time'].value_counts()[1])
vendor_otd_sing_full

'88.65%'

In [63]:
# get get defects data
# production env
# connect to SQL
db = 'Quality'
connection = pyodbc.connect('Driver={SQL Server}; Server=okcpsql01p; Database=' + db + '; uid=; pwd=')
cursor = connection.cursor()

sql_cmd = """
    SELECT *
    FROM [Quality].[dbo].[tbl_NCRData]
"""

df_defects = pd.read_sql(sql_cmd, connection)

df_defects.dtypes

_id                    object
Field                  object
Plnt                   object
Notification           object
Typ                    object
Stat                   object
Material               object
MaterialDescription    object
Description            object
Createdby              object
AD                     object
Referenceno            object
Vendor                 object
Name1                  object
Prodorder              object
Createdon              object
Complaintqty           object
DefectQty_int_         object
Defqty_ext_            object
Completion             object
Shorttextforcode       object
Shorttextforcode_1     object
Date_                  object
Field_1                object
Rpt_Date               object
id                      int64
CodeGroup              object
Task                   object
TaskText               object
PurchDoc               object
PurchItem              object
dtype: object

In [198]:
# get defects - had to read in excel as NCR tables not working for moment.
df_defects = pd.read_excel('ncr_data.xlsx', index='Notification')
df_defects

,Exception,Selected line,Notification type,Notification,Created on,Required Start,Required End,Notification Status,Priority text,Description,...,Code group,Client,City,Changed on,Changed by,Catalog profile,Catalog coding,Batch,Address number,Addit. device data
0,NaN,NaN,Z2,210179138.0,2017-01-19,2017-01-19,NaT,NOCO NOTE ATCO NOPT,NaN,Grease Leakage,...,CPAR,100.0,MUSKEGON,2017-08-28,174412,QM0000002,D,NaN,NaN,NaN
1,NaN,NaN,Z2,210184449.0,2017-02-21,2017-02-21,NaT,NOCO NOTE ATCO NOPT,NaN,PINHOLE IN BODY CASTING,...,CPAR,100.0,JIANGSU,2017-09-27,174412,QM0000002,D,NaN,NaN,NaN
2,NaN,NaN,Z2,210185743.0,2017-02-28,2017-02-28,NaT,NOCO NOTE ATCO NOPT,NaN,Casting defects in disc,...,CPAR,100.0,CORSICANA,2017-09-27,174412,QM0000002,D,NaN,NaN,NaN
3,NaN,NaN,Z2,210192189.0,2017-04-10,2017-04-10,NaT,OSNO NOTE OSTS NOPT,NaN,High volume of scrap due to porosity,...,CPAR,100.0,HOUSTON,2017-09-27,174412,QM0000002,D,NaN,NaN,NaN
4,NaN,NaN,Z2,210192700.0,2017-04-12,2017-04-12,NaT,OSNO NOTE OSTS NOPT,NaN,High volume of foundry error; cast issue,...,CPAR,100.0,TULSA,2017-09-27,174412,QM0000002,D,NaN,NaN,NaN
5,NaN,NaN,Z2,210192732.0,2017-04-12,2017-04-12,NaT,NOCO NOTE ATCO NOPT,NaN,High volume of rusted parts received,...,CPAR,100.0,HSINCHU COUNTY,2017-06-11,CHENJJ,QM0000002,D,NaN,NaN,NaN
6,NaN,NaN,Z2,210192733.0,2017-04-12,2017-04-12,NaT,OSNO NOTE OSTS NOPT,NaN,Surface finish issues for various sizes,...,CPAR,100.0,WENZHOU CITY,2017-09-27,174412,QM0000002,D,NaN,NaN,NaN
7,NaN,NaN,Z2,210192734.0,2017-04-12,2017-04-12,NaT,NOCO NOTE ATCO NOPT,NaN,High volume of undersized threads,...,CPAR,100.0,OKLAHOMA CITY,2017-08-28,174412,QM0000002,D,NaN,NaN,NaN
8,NaN,NaN,Z2,210195774.0,2017-05-04,2017-05-04,NaT,NOCO NOTE OSTS DLFL,NaN,Wrong thread,...,CPAR,100.0,AURORA,2017-09-29,169440,QM0000002,D,NaN,NaN,NaN
9,NaN,NaN,Z2,210197636.0,2017-05-16,2017-05-16,NaT,NOCO NOTE OSTS DLFL,NaN,Parts not to new revision level,...,CPAR,100.0,SEALY,2017-09-29,169440,QM0000002,D,NaN,NaN,NaN


In [199]:
df_defects.columns

Index(['Exception', 'Selected line', 'Notification type', 'Notification',
       'Created on', 'Required Start', 'Required End', 'Notification Status',
       'Priority text', 'Description',
       ...
       'Code group', 'Client', 'City', 'Changed on', 'Changed by',
       'Catalog profile', 'Catalog coding', 'Batch', 'Address number',
       'Addit. device data'],
      dtype='object', length=116)

In [196]:
df_defects.fillna(0)

,Exception,Selected line,Notification type,Notification,Created on,Required Start,Required End,Notification Status,Priority text,Description,...,Code group,Client,City,Changed on,Changed by,Catalog profile,Catalog coding,Batch,Address number,Addit. device data
0,0.0,0.0,Z2,210179138.0,2017-01-19,2017-01-19,1970-01-01,NOCO NOTE ATCO NOPT,0,Grease Leakage,...,CPAR,100.0,MUSKEGON,2017-08-28,174412,QM0000002,D,0.0,0.0,0
1,0.0,0.0,Z2,210184449.0,2017-02-21,2017-02-21,1970-01-01,NOCO NOTE ATCO NOPT,0,PINHOLE IN BODY CASTING,...,CPAR,100.0,JIANGSU,2017-09-27,174412,QM0000002,D,0.0,0.0,0
2,0.0,0.0,Z2,210185743.0,2017-02-28,2017-02-28,1970-01-01,NOCO NOTE ATCO NOPT,0,Casting defects in disc,...,CPAR,100.0,CORSICANA,2017-09-27,174412,QM0000002,D,0.0,0.0,0
3,0.0,0.0,Z2,210192189.0,2017-04-10,2017-04-10,1970-01-01,OSNO NOTE OSTS NOPT,0,High volume of scrap due to porosity,...,CPAR,100.0,HOUSTON,2017-09-27,174412,QM0000002,D,0.0,0.0,0
4,0.0,0.0,Z2,210192700.0,2017-04-12,2017-04-12,1970-01-01,OSNO NOTE OSTS NOPT,0,High volume of foundry error; cast issue,...,CPAR,100.0,TULSA,2017-09-27,174412,QM0000002,D,0.0,0.0,0
5,0.0,0.0,Z2,210192732.0,2017-04-12,2017-04-12,1970-01-01,NOCO NOTE ATCO NOPT,0,High volume of rusted parts received,...,CPAR,100.0,HSINCHU COUNTY,2017-06-11,CHENJJ,QM0000002,D,0.0,0.0,0
6,0.0,0.0,Z2,210192733.0,2017-04-12,2017-04-12,1970-01-01,OSNO NOTE OSTS NOPT,0,Surface finish issues for various sizes,...,CPAR,100.0,WENZHOU CITY,2017-09-27,174412,QM0000002,D,0.0,0.0,0
7,0.0,0.0,Z2,210192734.0,2017-04-12,2017-04-12,1970-01-01,NOCO NOTE ATCO NOPT,0,High volume of undersized threads,...,CPAR,100.0,OKLAHOMA CITY,2017-08-28,174412,QM0000002,D,0.0,0.0,0
8,0.0,0.0,Z2,210195774.0,2017-05-04,2017-05-04,1970-01-01,NOCO NOTE OSTS DLFL,0,Wrong thread,...,CPAR,100.0,AURORA,2017-09-29,169440,QM0000002,D,0.0,0.0,0
9,0.0,0.0,Z2,210197636.0,2017-05-16,2017-05-16,1970-01-01,NOCO NOTE OSTS DLFL,0,Parts not to new revision level,...,CPAR,100.0,SEALY,2017-09-29,169440,QM0000002,D,0.0,0.0,0


In [ ]:
# total rejected items
total_defects = 

In [200]:
df_defects_single = df_defects[df_defects['Vendor'].str.contains(vendor_code)]
df_defects_single

ValueError: cannot index with vector containing NA / NaN values

In [ ]:
# use docx to create word doc report
doc = docx.Document()

# get all pics in current directory
pic_files = []
for filename in os.listdir('.'):
    if filename.endswith('.png'):
        pic_files.append(filename)
       
doc.add_heading('Scrap Monthly Analysis - ' + rpt_date, 1)
for pic in pic_files:
    doc.add_picture(pic, width=docx.shared.Inches(6))
    
# doc.ad_page_break()

doc.save('myreport.docx')